In [1]:
!pip install keras-tcn
!pip install -U finance-datareader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

/home/work/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2023-01-31 02:58:33.359249: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-31 02:58:33.497848: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-31 02:58:36.411639: W tensorflow/compiler/xla/stream_executor/platform/default/

In [3]:
kospi_train = fdr.DataReader('KS11', '1990-01-01','2017-12-31')['Close']
print(kospi_train.head())
print(kospi_train)
print(kospi_train.shape)
print(kospi_train.isnull().any())

Date
1996-12-11    704.679993
1996-12-12    689.380005
1996-12-13    689.070007
1996-12-16    673.919983
1996-12-17    663.349976
Name: Close, dtype: float64
Date
1996-12-11     704.679993
1996-12-12     689.380005
1996-12-13     689.070007
1996-12-16     673.919983
1996-12-17     663.349976
                 ...     
2017-12-21    2429.830078
2017-12-22    2440.540039
2017-12-26    2427.340088
2017-12-27    2436.669922
2017-12-28    2467.489990
Name: Close, Length: 5341, dtype: float64
(5341,)
True


In [4]:
kospi_test = fdr.DataReader('KS11', '2018-01-01')['Close']
print(kospi_test)
print(kospi_test.shape)
print(kospi_test.isnull().any())

Date
2018-01-02    2479.649902
2018-01-03    2486.350098
2018-01-04    2466.459961
2018-01-05    2497.520020
2018-01-08    2513.280029
                 ...     
2023-01-25    2428.570068
2023-01-26    2468.649902
2023-01-27    2484.020020
2023-01-30    2450.469971
2023-01-31    2441.229980
Name: Close, Length: 1250, dtype: float64
(1250,)
False


In [5]:
kospi_train = kospi_train.dropna()
kospi_test = kospi_test.dropna()

print(kospi_train)
print(kospi_train.isnull().any())
print(kospi_test)
print(kospi_test.isnull().any())

Date
1996-12-11     704.679993
1996-12-12     689.380005
1996-12-13     689.070007
1996-12-16     673.919983
1996-12-17     663.349976
                 ...     
2017-12-21    2429.830078
2017-12-22    2440.540039
2017-12-26    2427.340088
2017-12-27    2436.669922
2017-12-28    2467.489990
Name: Close, Length: 5192, dtype: float64
False
Date
2018-01-02    2479.649902
2018-01-03    2486.350098
2018-01-04    2466.459961
2018-01-05    2497.520020
2018-01-08    2513.280029
                 ...     
2023-01-25    2428.570068
2023-01-26    2468.649902
2023-01-27    2484.020020
2023-01-30    2450.469971
2023-01-31    2441.229980
Name: Close, Length: 1250, dtype: float64
False


In [6]:
kospi_train = kospi_train.values
kospi_test = kospi_test.values

In [7]:
lookback_window = 60
x_train_1, y_train_1 = [], []
x_train_7, y_train_7 = [], []
x_train_15, y_train_15 = [], []
x_train_30, y_train_30 = [], []

In [8]:
for i in range(lookback_window, len(kospi_train)-1):
    x_train_1.append(kospi_train[i - lookback_window:i])
    y_train_1.append(kospi_train[i] - kospi_train[i-1])

for i in range(lookback_window, len(kospi_train)-6):
    x_train_7.append(kospi_train[i - lookback_window:i])
    y_train_7.append(kospi_train[i+6] - kospi_train[i-1])

for i in range(lookback_window, len(kospi_train)-14):
    x_train_15.append(kospi_train[i - lookback_window:i])
    y_train_15.append(kospi_train[i+14] - kospi_train[i-1])

for i in range(lookback_window, len(kospi_train)-29):
    x_train_30.append(kospi_train[i - lookback_window:i])
    y_train_30.append(kospi_train[i+29] - kospi_train[i-1])

In [9]:
for index in range(len(y_train_1)):
    if y_train_1[index]>= -10 and y_train_1[index] < 10:
        y_train_1[index] = 'neutral'
    elif y_train_1[index] >= 10:
        y_train_1[index] = 'increase'
    else:
        y_train_1[index] = 'decrease'
        
for index in range(len(y_train_7)):
    if y_train_7[index]>= -10 and y_train_7[index] < 10:
        y_train_7[index] = 'neutral'
    elif y_train_7[index] >= 10:
        y_train_7[index] = 'increase'
    else:
        y_train_7[index] = 'decrease'

for index in range(len(y_train_15)):
    if y_train_15[index]>= -10 and y_train_15[index] < 10:
        y_train_15[index] = 'neutral'
    elif y_train_15[index] >= 10:
        y_train_15[index] = 'increase'
    else:
        y_train_15[index] = 'decrease'

for index in range(len(y_train_30)):
    if y_train_30[index]>= -10 and y_train_30[index] < 10:
        y_train_30[index] = 'neutral'
    elif y_train_30[index] >= 10:
        y_train_30[index] = 'increase'
    else:
        y_train_30[index] = 'decrease'

In [10]:
x_train_1 = np.array(x_train_1)
y_train_1 = np.array(y_train_1)
x_train_1 = x_train_1.reshape((5131, -1, 1))
#y_train_1 = y_train_1.reshape((-1, 1))

x_train_7 = np.array(x_train_7)
y_train_7 = np.array(y_train_7)
x_train_7 = x_train_7.reshape((5126, -1, 1))
#y_train_7 = y_train_7.reshape((-1, 1))

x_train_15 = np.array(x_train_15)
y_train_15 = np.array(y_train_15)
x_train_15 = x_train_15.reshape((5118, -1, 1))
#y_train_15 = y_train_15.reshape((-1, 1))

x_train_30 = np.array(x_train_30)
y_train_30 = np.array(y_train_30)
x_train_30 = x_train_30.reshape((5103, -1, 1))
#y_train_30 = y_train_30.reshape((-1, 1))

print(x_train_1.shape, x_train_7.shape, x_train_15.shape, x_train_30.shape)
print(y_train_1.shape, y_train_7.shape, y_train_15.shape, y_train_30.shape)

(5131, 60, 1) (5126, 60, 1) (5118, 60, 1) (5103, 60, 1)
(5131,) (5126,) (5118,) (5103,)


In [11]:
x_test_1, y_test_1 = [], []
x_test_7, y_test_7 = [], []
x_test_15, y_test_15 = [], []
x_test_30, y_test_30 = [], []

In [12]:
for i in range(lookback_window, len(kospi_test)):
    x_test_1.append(kospi_test[i - lookback_window:i])
    y_test_1.append(kospi_test[i] - kospi_test[i-1])

for i in range(lookback_window, len(kospi_test)-6):
    x_test_7.append(kospi_test[i - lookback_window:i])
    y_test_7.append(kospi_test[i+6] - kospi_test[i-1])

for i in range(lookback_window, len(kospi_test)-14):
    x_test_15.append(kospi_test[i - lookback_window:i])
    y_test_15.append(kospi_test[i+14] - kospi_test[i-1])

for i in range(lookback_window, len(kospi_test)-29):
    x_test_30.append(kospi_test[i - lookback_window:i])
    y_test_30.append(kospi_test[i+29] - kospi_test[i-1])

In [13]:
for index in range(len(y_test_1)):
    if y_test_1[index]>= -10 and y_test_1[index] < 10:
        y_test_1[index] = 'neutral'
    elif y_test_1[index] >= 10:
        y_test_1[index] = 'increase'
    else:
        y_test_1[index] = 'decrease'
        
for index in range(len(y_test_7)):
    if y_test_7[index]>= -10 and y_test_7[index] < 10:
        y_test_7[index] = 'neutral'
    elif y_test_7[index] >= 10:
        y_test_7[index] = 'increase'
    else:
        y_test_7[index] = 'decrease'

for index in range(len(y_test_15)):
    if y_test_15[index]>= -10 and y_test_15[index] < 10:
        y_test_15[index] = 'neutral'
    elif y_test_15[index] >= 10:
        y_test_15[index] = 'increase'
    else:
        y_test_15[index] = 'decrease'

for index in range(len(y_test_30)):
    if y_test_30[index]>= -10 and y_test_30[index] < 10:
        y_test_30[index] = 'neutral'
    elif y_test_30[index] >= 10:
        y_test_30[index] = 'increase'
    else:
        y_test_30[index] = 'decrease'

In [17]:
x_test_1 = np.array(x_test_1)
y_test_1 = np.array(y_test_1)
x_test_1 = x_test_1.reshape((1190, -1, 1))
#y_test_1 = y_test_1.reshape((-1, 1))

x_test_7 = np.array(x_test_7)
y_test_7 = np.array(y_test_7)
x_test_7 = x_test_7.reshape((1184, -1, 1))
#y_test_7 = y_test_7.reshape((-1, 1))

x_test_15 = np.array(x_test_15)
y_test_15 = np.array(y_test_15)
x_test_15 = x_test_15.reshape((1176, -1, 1))
#y_test_15 = y_test_15.reshape((-1, 1))

x_test_30 = np.array(x_test_30)
y_test_30 = np.array(y_test_30)
x_test_30 = x_test_30.reshape((1161, -1, 1))
#y_test_30 = y_test_30.reshape((-1, 1))

In [18]:
print(x_test_1.shape, x_test_7.shape, x_test_15.shape, x_test_30.shape)
print(y_test_1.shape, y_test_7.shape, y_test_15.shape, y_test_30.shape)

(1190, 60, 1) (1184, 60, 1) (1176, 60, 1) (1161, 60, 1)
(1190,) (1184,) (1176,) (1161,)


In [19]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y_train_1)

y_train_1_e = to_categorical(encoder.transform(y_train_1))
y_train_7_e = to_categorical(encoder.transform(y_train_7))
y_train_15_e = to_categorical(encoder.transform(y_train_15))
y_train_30_e = to_categorical(encoder.transform(y_train_30))

y_test_1_e = to_categorical(encoder.transform(y_test_1))
y_test_7_e = to_categorical(encoder.transform(y_test_7))
y_test_15_e = to_categorical(encoder.transform(y_test_15))
y_test_30_e = to_categorical(encoder.transform(y_test_30))

In [20]:
model_1 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_1.summary()
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_7 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_7.summary()
model_7.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_15 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_15.summary()
model_15.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_30 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_30.summary()
model_30.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-01-31 02:59:26.155579: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/nvidia/lib64:/usr/include/x86_64-linux-gnu
2023-01-31 02:59:26.155643: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-31 02:59:26.155691: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2023-01-31 02:59:26.156282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in othe

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn (TCN)                   (None, 64)                91136     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 91,331
Trainable params: 91,331
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_1 (TCN)                 (None, 64)                91136     
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 91,331
Trainable

In [21]:
print('Train model_1...')
model_1.fit(x_train_1, y_train_1_e, epochs=100, verbose=1)
p_test_1 = model_1.predict(x_test_1)

print('Train model_7...')
model_7.fit(x_train_7, y_train_7_e, epochs=100, verbose=1)
p_test_7 = model_7.predict(x_test_7)

print('Train model_15...')
model_15.fit(x_train_15, y_train_15_e, epochs=100, verbose=1)
p_test_15 = model_15.predict(x_test_15)

print('Train model_30...')
model_30.fit(x_train_30, y_train_30_e, epochs=100, verbose=1)
p_test_30 = model_30.predict(x_test_30)

Train model_1...
Epoch 1/100
161/161 [==============================] - 6s 19ms/step - loss: 1263.3472 - accuracy: 0.3912
Epoch 2/100
161/161 [==============================] - 3s 18ms/step - loss: 306.1275 - accuracy: 0.3894
Epoch 3/100
161/161 [==============================] - 3s 18ms/step - loss: 290.9110 - accuracy: 0.3806
Epoch 4/100
161/161 [==============================] - 3s 17ms/step - loss: 115.4473 - accuracy: 0.3952
Epoch 5/100
161/161 [==============================] - 3s 16ms/step - loss: 148.3853 - accuracy: 0.3820
Epoch 6/100
161/161 [==============================] - 3s 17ms/step - loss: 95.1834 - accuracy: 0.3991
Epoch 7/100
161/161 [==============================] - 3s 18ms/step - loss: 112.7346 - accuracy: 0.3974
Epoch 8/100
161/161 [==============================] - 3s 16ms/step - loss: 76.4622 - accuracy: 0.3968
Epoch 9/100
161/161 [==============================] - 3s 16ms/step - loss: 68.0676 - accuracy: 0.3972
Epoch 10/100
161/161 [===========================

161/161 [==============================] - 3s 16ms/step - loss: 6.3486 - accuracy: 0.4030
Epoch 80/100
161/161 [==============================] - 3s 16ms/step - loss: 6.8085 - accuracy: 0.3972
Epoch 81/100
161/161 [==============================] - 3s 16ms/step - loss: 6.3832 - accuracy: 0.3993
Epoch 82/100
161/161 [==============================] - 3s 16ms/step - loss: 5.3480 - accuracy: 0.4067
Epoch 83/100
161/161 [==============================] - 3s 16ms/step - loss: 7.6159 - accuracy: 0.3880
Epoch 84/100
161/161 [==============================] - 3s 16ms/step - loss: 6.2811 - accuracy: 0.3902
Epoch 85/100
161/161 [==============================] - 3s 16ms/step - loss: 7.4372 - accuracy: 0.3855
Epoch 86/100
161/161 [==============================] - 3s 16ms/step - loss: 6.2208 - accuracy: 0.3837
Epoch 87/100
161/161 [==============================] - 3s 16ms/step - loss: 5.8564 - accuracy: 0.3929
Epoch 88/100
161/161 [==============================] - 3s 16ms/step - loss: 5.4142 - 

161/161 [==============================] - 4s 26ms/step - loss: 15.9389 - accuracy: 0.3595
Epoch 58/100
161/161 [==============================] - 4s 26ms/step - loss: 19.2748 - accuracy: 0.3712
Epoch 59/100
161/161 [==============================] - 4s 26ms/step - loss: 12.7730 - accuracy: 0.3658
Epoch 60/100
161/161 [==============================] - 4s 26ms/step - loss: 15.6489 - accuracy: 0.3658
Epoch 61/100
161/161 [==============================] - 4s 26ms/step - loss: 12.1039 - accuracy: 0.3876
Epoch 62/100
161/161 [==============================] - 4s 26ms/step - loss: 12.5771 - accuracy: 0.3691
Epoch 63/100
161/161 [==============================] - 4s 26ms/step - loss: 13.6135 - accuracy: 0.3818
Epoch 64/100
161/161 [==============================] - 4s 26ms/step - loss: 13.9759 - accuracy: 0.3679
Epoch 65/100
161/161 [==============================] - 4s 26ms/step - loss: 12.8307 - accuracy: 0.3650
Epoch 66/100
161/161 [==============================] - 4s 26ms/step - loss: 

160/160 [==============================] - 4s 26ms/step - loss: 39.9480 - accuracy: 0.4158
Epoch 36/100
160/160 [==============================] - 4s 26ms/step - loss: 41.6549 - accuracy: 0.4109
Epoch 37/100
160/160 [==============================] - 4s 24ms/step - loss: 23.9780 - accuracy: 0.4117
Epoch 38/100
160/160 [==============================] - 4s 23ms/step - loss: 27.8416 - accuracy: 0.4162
Epoch 39/100
160/160 [==============================] - 4s 26ms/step - loss: 28.3264 - accuracy: 0.4078
Epoch 40/100
160/160 [==============================] - 4s 26ms/step - loss: 33.7278 - accuracy: 0.4175
Epoch 41/100
160/160 [==============================] - 4s 26ms/step - loss: 38.4108 - accuracy: 0.4144
Epoch 42/100
160/160 [==============================] - 4s 26ms/step - loss: 32.8645 - accuracy: 0.4074
Epoch 43/100
160/160 [==============================] - 4s 26ms/step - loss: 23.9158 - accuracy: 0.4089
Epoch 44/100
160/160 [==============================] - 4s 26ms/step - loss: 

160/160 [==============================] - 4s 26ms/step - loss: 45.8307 - accuracy: 0.4680
Epoch 14/100
160/160 [==============================] - 4s 26ms/step - loss: 43.7670 - accuracy: 0.4456
Epoch 15/100
160/160 [==============================] - 4s 26ms/step - loss: 53.3488 - accuracy: 0.4560
Epoch 16/100
160/160 [==============================] - 4s 26ms/step - loss: 37.7990 - accuracy: 0.4460
Epoch 17/100
160/160 [==============================] - 4s 26ms/step - loss: 31.5257 - accuracy: 0.4568
Epoch 18/100
160/160 [==============================] - 4s 26ms/step - loss: 37.9613 - accuracy: 0.4560
Epoch 19/100
160/160 [==============================] - 4s 26ms/step - loss: 34.9914 - accuracy: 0.4531
Epoch 20/100
160/160 [==============================] - 4s 26ms/step - loss: 27.9972 - accuracy: 0.4540
Epoch 21/100
160/160 [==============================] - 4s 26ms/step - loss: 30.0813 - accuracy: 0.4419
Epoch 22/100
160/160 [==============================] - 4s 26ms/step - loss: 

160/160 [==============================] - 4s 27ms/step - loss: 4.1914 - accuracy: 0.4411
Epoch 93/100
160/160 [==============================] - 4s 26ms/step - loss: 3.3153 - accuracy: 0.4435
Epoch 94/100
160/160 [==============================] - 4s 26ms/step - loss: 5.0106 - accuracy: 0.4368
Epoch 95/100
160/160 [==============================] - 4s 27ms/step - loss: 5.9800 - accuracy: 0.4407
Epoch 96/100
160/160 [==============================] - 4s 26ms/step - loss: 4.1403 - accuracy: 0.4623
Epoch 97/100
160/160 [==============================] - 4s 26ms/step - loss: 4.3023 - accuracy: 0.4384
Epoch 98/100
160/160 [==============================] - 4s 26ms/step - loss: 4.9221 - accuracy: 0.4384
Epoch 99/100
160/160 [==============================] - 4s 26ms/step - loss: 5.7082 - accuracy: 0.4486
Epoch 100/100
37/37 [==============================] - 1s 8ms/step


In [22]:
model_1.save('/home/work/nlp/TCN_models/classification_model/v3/kospi_1day_classification_v3.h5')
model_7.save('/home/work/nlp/TCN_models/classification_model/v3/kospi_7days_classification_v3.h5')
model_15.save('/home/work/nlp/TCN_models/classification_model/v3/kospi_15days_classification_v3.h5')
model_30.save('/home/work/nlp/TCN_models/classification_model/v3/kospi_30days_classification_v3.h5')

In [23]:
print('\n Accuracy: %.4f' % (model_1.evaluate(x_test_1, y_test_1_e)[1]))
print('\n Accuracy: %.4f' % (model_7.evaluate(x_test_7, y_test_7_e)[1]))
print('\n Accuracy: %.4f' % (model_15.evaluate(x_test_15, y_test_15_e)[1]))
print('\n Accuracy: %.4f' % (model_30.evaluate(x_test_30, y_test_30_e)[1]))

38/38 [==============================] - 1s 25ms/step - loss: 4.7377 - accuracy: 0.3538

 Accuracy: 0.3538
37/37 [==============================] - 1s 25ms/step - loss: 31.6821 - accuracy: 0.4088

 Accuracy: 0.4088
37/37 [==============================] - 1s 25ms/step - loss: 22.4626 - accuracy: 0.4609

 Accuracy: 0.4609
37/37 [==============================] - 1s 25ms/step - loss: 26.4980 - accuracy: 0.4729

 Accuracy: 0.4729
